In [ ]:
!pip install oauthlib
!pip install requests_oauthlib
!pip install sentinelhub

In [2]:
import pandas as pd
from PIL import Image
from datetime import datetime, timedelta
import requests
from data_mining.get_sat2_images import call_sentinel
from data_mining.read_firenews import fetch_fire_news
import data_mining.read_weather as weather

# read fire history

## fetch from fire news

In [ ]:
start_date="2022-05-01"
end_date="2025-06-10"
fire_data=fetch_fire_news(start_date,end_date,verbose=True)
# fetch_fire_news(start_date,end_date,save=True)

## or read from file

In [8]:
# fire_data= pd.read_csv("fire_news.csv")
fire_data = pd.read_csv("fire_confidence/fire_confidence_clean.csv")
fire_data = fire_data[["latitude", "longitude", "date"]][::-1]
# print(fire_data)

## genrate dates for the sentences of the RNN

In [ ]:
fire_month=pd.DataFrame(columns=fire_data.columns)
for index, row in fire_data.iterrows():
    day = row["date"]
    day_date = datetime.strptime(day, "%Y-%m-%d")
    # dates = [(day_date + timedelta(days=i)).strftime("%Y-%m-%d") for i in range(-15, 16)] # get 30 days around that date
    # dates = [(day_date + timedelta(weeks=i)).strftime("%Y-%m-%d") for i in range(-2, 2)]  # sample every week for a month around that date
    dates = [(day_date + timedelta(days=i)).strftime("%Y-%m-%d") for i in range(-7, 2)]  # sample every day for a week
    month=[[row['latitude'],row['longitude'],date] for date in dates]
    month = pd.DataFrame(month, columns=fire_data.columns)
    fire_month = pd.concat([fire_month,month], ignore_index=True)

# get images from sentinel 2

In [9]:
# Your client credentials
client_id = '4d093505-d867-4ae4-90b0-468eb78dd9af'
client_secret = 'UHSroMy3xUHqh0SakSXrPvCejglb2eyH'

In [10]:
for index, row in fire_data.iterrows():
    day = row["date"]
    day_date = datetime.strptime(day, "%Y-%m-%d")
    yesterday = (day_date - timedelta(days=1)).strftime("%Y-%m-%d")
    time_interval=(yesterday, day)
    sucsess = call_sentinel(client_id,client_secret,row,time_interval,save=True)
    print(f"sucsess {sucsess} on {day} in {row["latitude"], row['longitude']}")
    if sucsess:
        dates = [(day_date + timedelta(days=i)).strftime("%Y-%m-%d") for i in range(-7, 2)]  # sample every day for a week
        print(dates)
        for date in dates:
            day_date = datetime.strptime(date, "%Y-%m-%d")
            yesterday = (day_date - timedelta(days=1)).strftime("%Y-%m-%d")
            time_interval=(yesterday, date)
            call_sentinel(client_id,client_secret,row,time_interval,save=True)
            print(f"sucsess {sucsess} on {date} in {row["latitude"], row['longitude']}")
            

sucsess True on 2024-06-09 in (40.38, 18.08)
['2024-06-02', '2024-06-03', '2024-06-04', '2024-06-05', '2024-06-06', '2024-06-07', '2024-06-08', '2024-06-09', '2024-06-10']
Picture not found for location 40.38, 18.08 on 2024-06-02
sucsess True on 2024-06-02 in (40.38, 18.08)
Picture not found for location 40.38, 18.08 on 2024-06-03
sucsess True on 2024-06-03 in (40.38, 18.08)
sucsess True on 2024-06-04 in (40.38, 18.08)
sucsess True on 2024-06-05 in (40.38, 18.08)
sucsess True on 2024-06-06 in (40.38, 18.08)
sucsess True on 2024-06-07 in (40.38, 18.08)
Picture not found for location 40.38, 18.08 on 2024-06-08
sucsess True on 2024-06-08 in (40.38, 18.08)
sucsess True on 2024-06-09 in (40.38, 18.08)
sucsess True on 2024-06-10 in (40.38, 18.08)
Picture not found for location 40.57, 17.37 on 2024-06-08
sucsess False on 2024-06-08 in (40.57, 17.37)
Picture not found for location 40.43, 17.39 on 2024-06-08
sucsess False on 2024-06-08 in (40.43, 17.39)
Picture not found for location 40.43, 17.

# get weather data

## only for dates where we found picture

In [ ]:
import os
import pandas as pd
# Specify the directory path
directory_path = 'sentinel_images'  # Replace with your directory path
# Get a list of all entries (files and subdirectories) in the specified directory
entries = os.listdir(directory_path)
# Iterate over each entry
rows=[]
for entry in entries:
    lat,long=entry.split(",")
    # Create the full path to the entry
    full_path = os.path.join(directory_path, entry)

    # Check if the entry is a directory
    if os.path.isdir(full_path):
        print(f"Subfolder: {entry}")

        # List all files and subfolders within the subfolder
        dates = os.listdir(full_path)
        for date in dates:
            rows.append([lat,long,date.split(".")[0]])
            print(f"  - {date}")
    else:
        print(f"File: {entry}")

fire_weather = pd.DataFrame(rows, columns=["lat","long","date"])

In [ ]:
openmeteo_session=weather.start_openmeteo_session()
for index, row in fire_weather.iterrows():
    day = row["date"]
    day_date = datetime.strptime(day, "%Y-%m-%d")
    yesterday = (day_date - timedelta(days=1)).strftime("%Y-%m-%d")
    time_interval=(yesterday, day)
    weather.get_weather(openmeteo_session,row,time_interval,verbose=False)